In [45]:
import os

from dotenv import load_dotenv
from openai import OpenAI
from groq import Groq

import gradio as gr


In [46]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GEMINI_BASE = "https://generativelanguage.googleapis.com/v1beta/openai/"
GROQ_BASE = "https://api.groq.com/openai/v1"

groq_client = OpenAI(api_key=GROQ_API_KEY, base_url=GROQ_BASE)
google_client = OpenAI(api_key=GOOGLE_API_KEY, base_url=GEMINI_BASE)

Task_Testing = "Generate Unit Tests"
Task_DocString = "Add DocString Comments"

In [47]:
def user_prompt_for(CSharp_Code, task):
    f"Rewrite this {CSharp_Code} using Python language. Code should be executable and free of errors. " + \
    f"Respond only in Python language without any errors. Also do the following based on subsequent {task}"
    if task == Task_Testing:
        return f"Generate unit tests for the generated code:\n\n{CSharp_Code}"
    elif task == Task_DocString:
        return f"Add docstring comments to the following generated code:\n\n{CSharp_Code}"
    else:
        raise ValueError("Unknown task")

In [48]:
def messages_for(CSharp_Code, task):
    system_message = f"You are a Developer assistant that converts CSharp code to Python code. Follow Python best practices to ensure the generated code is efficient and clean. Output block should be only Python code without any explanations."
    if task == Task_Testing:
        system_message += "You will generate unit tests for the provided code."
    elif task == Task_DocString:
        system_message += "You will add docstring comments to the provided code."
    
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(CSharp_Code, task)}
    ]

In [49]:
def run_code_generation(CSharp_Code, task, model_choice): 
    if model_choice == "Groq": 
        client = groq_client 
        model = "openai/gpt-oss-20b" 
    else: 
        client = google_client 
        model = "gemini-2.5-flash" 
    messages = messages_for(CSharp_Code, task) 
    response = client.chat.completions.create( model=model, 
                messages=messages, 
                stream=False ) 
    return response.choices[0].message.content

In [50]:
CSharpCode=""" --Sample CSharp Code
using System;
using System.Collections.Generic;
using System.Linq;

namespace CodeGenSample
{
    public enum AccountType
    {
        Savings,
        Checking
    }

    public class BankAccount
    {
        public string AccountNumber { get; }
        public string OwnerName { get; }
        public AccountType Type { get; }
        public decimal Balance { get; private set; }

        public BankAccount(string accountNumber, string ownerName, AccountType type)
        {
            if (string.IsNullOrWhiteSpace(accountNumber))
                throw new ArgumentException("Account number is required.");

            AccountNumber = accountNumber;
            OwnerName = ownerName;
            Type = type;
            Balance = 0m;
        }

        public void Deposit(decimal amount)
        {
            if (amount <= 0)
                throw new ArgumentException("Deposit amount must be positive.");

            Balance += amount;
        }

        public void Withdraw(decimal amount)
        {
            if (amount <= 0)
                throw new ArgumentException("Withdrawal amount must be positive.");

            if (amount > Balance)
                throw new InvalidOperationException("Insufficient balance.");

            Balance -= amount;
        }
    }

    public class BankService
    {
        private readonly List<BankAccount> _accounts = new();

        public void AddAccount(BankAccount account)
        {
            if (account == null)
                throw new ArgumentNullException(nameof(account));

            _accounts.Add(account);
        }

        public decimal GetTotalBalance(AccountType type)
        {
            return _accounts
                .Where(a => a.Type == type)
                .Sum(a => a.Balance);
        }

        public BankAccount FindAccount(string accountNumber)
        {
            return _accounts.FirstOrDefault(a => a.AccountNumber == accountNumber);
        }
    }
}

"""

In [51]:
with gr.Blocks() as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            csharp_code = gr.Code(
                label="CSharp (original)",
                value=CSharpCode,
                language=None,
            )
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (generated)",
                value="",
                language="python",
            )
    with gr.Row():
        model_choice = gr.Radio(
            label="Model Choice",
            choices=["Groq", "Gemini"],
            value="Groq",
            type="value",
            interactive=True,
        )
        task = gr.Radio(
            label="Task",
            choices=[Task_Testing, Task_DocString],
            value=Task_Testing,
            type="value",
            interactive=True,
        )
    generate_button = gr.Button("Generate Code")
    generate_button.click(
        fn=run_code_generation,
        inputs=[csharp_code, task, model_choice],
        outputs=[python],
    )
if __name__ == "__main__":
    ui.launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.
